# Rouge Score - Rouge(Recall-Oriented Understudy for Gisting Evaluation)

https://velog.io/@jochedda/Rouge-Score-Text-Summarization%EC%9D%98-%ED%8F%89%EA%B0%80%EC%A7%80%ED%91%9C


- 텍스트 요약 모델 성능 평가에 주로 사용되는 지표로 Label(사람이 만든 요약문)과 Summary(모델이 생성한 inference)를 비교해서 성능 계산

- ROUGE - N , ROUGE - L, ROUGE - W, ROUGE - S 등 다양한 지표가 존재함
각각 지표 별로 recall 및 precision을 둘 다 구하는 것이 좋다

- Recall
: label을 구성하는 단어 중 몇개가 inference 와 겹치는지 확인

    - 우선적으로 필요한 정보들이 다 담겨있는지 확인 !
- Precision
: inference를 구성하는 단어 중 몇개가 Label과 겹치는지 확인

    - 요약된 문장에 필요한 정보만을 얼마나 담고 있는지 확인 !

# ROUGE - N
- N-gram의 개수가 기준
- ROUGE - 1은 unigram, ROUGE - 2 는 bigram,…
- Recall: ouput과 겹치는 N-gram의 수 / Label의 N - gram의 수
- Precision : Label 과 겹치는 N-gram의 수 / output의 N-gram의 수

# ROUGE - L
- LCS(Longest common Sequence) between model output
- common sequence 중에서 가장 긴 것을 매칭
- N- gram과 달리 순서나 위치 관계를 고려한 알고리즘
- Recall : LCS 길이 / Label의 N-gram 수
- Precision : LCS 길이 / output의 N-gram 수

# ROUGE - S
- Skip - bigram을 활용한 metric
- 두 개의 토큰을 한 상으로 묵어서 ROUGE Score 계산
- 예를 들어, ‘ the brown fox’ 는 (the,brown),(brown,fox),(the,fox) 로 매핑되어 계산
- Window Size가 주어졌을 때, 이 사이즈 내에 위치하는 단어쌍들을 묶어 해당 단어쌍들이 얼마나 중복되는지 측정

# ROUGE Score의 단점
동음이의어에 대해서 평가할 수 없음 ! 즉, 같은 의미의 다른 단어가 많으면 성능을 낮게 측정한다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import random
import os

import time

!pip install rouge
from rouge import Rouge

import re

In [3]:
# 모델명과 폴더명과 일치시켜서
model_list = ['base_model', 'model1_3000', 'model1_10000', 'prompt10000', 'model_2bnochat']

In [4]:
for model in model_list:
    df = pd.DataFrame(columns = ['text', 'origin_summary', 'summary'])
    selected_csv = []

    folder_path = f'/content/drive/MyDrive/new_project/project2/test_result/{model}/'
    file_list = os.listdir(folder_path)
    selected_csv = [csv for csv in file_list if 'csv' in csv]
    selected_csv = sorted(selected_csv)

    for file_name in selected_csv[:5]:

        path = folder_path + file_name
        temp = pd.read_csv(path)

        df = pd.concat([df, temp], axis = 0)
        # df를 순서대로 입력하기 위해 -- 처리상 이전 df가 느리게 concat될수도
        time.sleep(0.2)

    df['summary'] = df['summary'].str.strip()
    df['modify_summary'] = df['summary']
    df.drop_duplicates(inplace=True)
    df.reset_index(drop = True, inplace=True)
    globals()[model] = df

In [5]:
for model in model_list:
    print(model, globals()[model].shape)

base_model (5000, 4)
model1_3000 (5000, 4)
model1_10000 (5000, 4)
prompt10000 (5000, 4)
model_2bnochat (1000, 4)


## 요약 결과 에러항 검출

- ROUGE - score의 정확한 판별
- 모델 선정 기준중 오류율 확인

In [6]:
## 특정 단어로 시작되는 에러
def error_start(model, word):
    print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
    print('error_word :', word)
    print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
    idx = model[model['summary'].str.contains(word)]['summary'].index[0]

    print('수정전 오류수: ', model['summary'].str.contains(word).sum())
    print('수정전 문장 \n: ', model['summary'][idx])
    print('*********************************************************************')

    model['modify_summary'] = model['modify_summary'].apply(lambda x: x.split(word)[0])
    model['modify_summary'] = model['modify_summary'].str.strip()

    print('수정후 오류수: ', model['modify_summary'].str.contains(word).sum())
    print('수정후 문장 \n: ', model['modify_summary'][idx])

## 특정 단어로 시작되는 에러 -> model3에 적용 / 뒷부분이 정답인 경우
def error_start2(model, word):
    print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
    print('error_word :', word)
    print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
    idx = model[model['summary'].str.contains(word)]['summary'].index[0]

    print('수정전 오류수: ', model['summary'].str.contains(word).sum())
    print('수정전 문장 \n: ', model['summary'][idx])
    print('*********************************************************************')

    model['modify_summary'] = model['modify_summary'].apply(lambda x: x.split(word)[0])
    model['modify_summary'] = model['modify_summary'].str.strip()

    print('수정후 오류수: ', model['modify_summary'].str.contains(word).sum())
    print('수정후 문장 \n: ', model['modify_summary'][idx])




In [7]:
## 특정 문구 제거
def delete_words(model, word):
    print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
    print('error_word :', word)
    print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')

    idx = model[model['summary'].str.contains(word)]['summary'].index[0]

    print('수정전 오류수: ', model['summary'].str.contains(word).sum())
    print('수정전 문장 \n: ', model['summary'][idx])

    print('*********************************************************************')

    model['modify_summary'] = model['modify_summary'].apply(lambda x: re.sub(word, '', x))
    model['modify_summary'] = model['modify_summary'].str.strip()

    print('수정후 오류수: ', model['modify_summary'].str.contains(word).sum())
    print('수정후 문장 \n: ', model['modify_summary'][idx])


In [8]:
## error 수정 후 영어만 있는 문장이 있다... 이를 거르기 위한 함수
def delete_nohangul(text):
    hangul_pattern = re.compile('[\u3131-\uD79D\uDC00-\uDEFC]+')
    return bool(hangul_pattern.search(text))



In [9]:
# 에러율 확인
def final_check(model):

    noerror = len(model[model['summary'] == model['modify_summary']])
    error = len(model[model['summary'] != model['modify_summary']])
    error_rate = error / (noerror + error)

    globals()[f'{model}_error'] = error
    globals()[f'{model}_noerror'] = noerror
    globals()[f'{model}_error_rate'] = error_rate

    print('정상출력 :', noerror)
    print('오류출력 :', error)
    print('오류률 :', error_rate * 100, '%')


## base_model 오류항 점검

In [10]:
# base_model

print('********************')
print('error type1') # Sure, here is a summary of the article in 100 characters:\n\n
print(base_model['summary'][4996])
print('********************')
print('error type2') # **Summary:** -> 뒷부분이 실제 요약
print(base_model['summary'][66])
print('********************')
print('error type3') # 영어로만 출력
print(base_model['summary'][233])
print('********************')
print('error type4') # (110 characters) -> 글자수 정보
print(base_model['summary'][345])
print('********************')
print('error type5') # </strong> -> html 태그
print(base_model['summary'][299])

********************
error type1
Sure, here is a summary of the article in 100 characters:

가천대 길병원, 최신 CT 장비 도입, 영상 선명도 개선, 피폭량 감소, 재협착 유무, 의식 없는 환자 촬영, 신규 CT 도입, 원스톱 검사 구현.
********************
error type2
**Summary:**

인테리어 비교견적 O2O 전문기업 집닥과 르호봇은 공유오피스 시장에 시너지 효과를 창출하기 위해 MOU를 체결했다.
********************
error type3
Sure, here's a summary of the article in about one sentence:

The article discusses the decline in government surplus funds and how it has impacted the economy, with the government's investment in infrastructure and housing leading to a significant drop in the amount of surplus funds available for discretionary spending.
********************
error type4
박유천 팬들이 박유천 지지 응원글을 전달했다.

(110 characters)
********************
error type5
블랙홀은 빛이 닿을 수 없는 거리에 있는 것으로 알려졌다. 

**(100자)</strong>


In [11]:
base_model_error_delete = ['Sure, here is a summary of the article in 100 characters:',  #type1
                           "Sure, here's a summary of the article in 100 characters:", #type1
                           '\*\*Summary:\*\*', #type2
                           "Sure, here's a summary of the article in about one sentence:", #type3
                           r'\(\d{3} characters\)', #type4 '(110 characters\)'
                           r'\(100자\)', #type4
                           '</strong>', #type5
                           '\*\*Note:\*\* 100 characters character limit\.' #type6
                           ]

base_model_error_split = []


In [12]:
# base_model
model0 = globals()[model_list[0]].copy()
print(model0.shape)

(5000, 4)


In [13]:
for word in base_model_error_delete:
    print('-------------------------------------------------------------\n\n')
    delete_words(model0, word)


-------------------------------------------------------------


@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
error_word : Sure, here is a summary of the article in 100 characters:
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
수정전 오류수:  167
수정전 문장 
:  Sure, here is a summary of the article in 100 characters:

Bio biotech company Carbiotex received positive opinions from external auditors for its financial statements for the past year, but its internal accounting management system received a negative opinion.
*********************************************************************
수정후 오류수:  0
수정후 문장 
:  Bio biotech company Carbiotex received positive opinions from external auditors for its financial statements for the past year, but its internal accounting management system received a negative opinion.
-------------------------------------------------------------


@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
error_wo

In [14]:
model0['modify_summary'] = model0['modify_summary'].apply(lambda x: x if delete_nohangul(x) == 1 else ' ')
print('error type3') # 영어로만 출력

error type3


In [15]:
final_check(model0)

정상출력 : 3994
오류출력 : 1006
오류률 : 20.119999999999997 %


# model1_3000 오류항 점검

In [16]:
model1_3000.head()


,text,origin_summary,summary,modify_summary
0,"[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, 체...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...,"문재인 대통령은 8일 서울 올림픽공원에서 열린 5G플러스 전략발표에 참석해 ""5G는...","문재인 대통령은 8일 서울 올림픽공원에서 열린 5G플러스 전략발표에 참석해 ""5G는..."
1,] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다. 바른미래당이 8일 연 ...,8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...,"바른미래당이 8일 연 최고위원회에서 손학규 대표와 김관영 원내대표만 참석한 가운데,...","바른미래당이 8일 연 최고위원회에서 손학규 대표와 김관영 원내대표만 참석한 가운데,..."
2,[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀. 지난 3일 한국이 세계 최초로 ...,지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...,5G 이동통신 서비스를 상용화한 것을 기념하는 '코리안 5G 테크-콘서트'가 8일 ...,5G 이동통신 서비스를 상용화한 것을 기념하는 '코리안 5G 테크-콘서트'가 8일 ...
3,] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...,"서울시장이 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석해 ""과거의...","서울시장이 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석해 ""과거의..."
4,"[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를 ...",주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...,KB자산운용은 코스닥 상장사 KMH에 이런 내용을 담은 주주서한을 최근 보냈다. K...,KB자산운용은 코스닥 상장사 KMH에 이런 내용을 담은 주주서한을 최근 보냈다. K...


In [17]:
print('********************')
print('error type1') # patrie: -> 뒷부분 제거
print(model1_3000['summary'][870])
print('********************')
print('error type2') # user -> 제거
print(model1_3000['summary'][543])
print('********************')
print('error type3') # model -> 뒷부분 제거
print(model1_3000['summary'][75])
print('********************')
print('error type4') # municipi -> 잘못된 단어  / 해당 단어가 포함된 단어를 제거
print(model1_3000['summary'][97])
print('********************')
print('error type5') # centrifuga -> 잘못된 단어  / 오류 시작
print(model1_3000['summary'][885])
print('********************')
print('error type6') # modelrehension · 100자 제출. -> 잘못된 단어  / 오류 시작
print(model1_3000['summary'][152])


********************
error type1
올해 KPGA 코리안투어 개막전인 'DB손해보험 프로미오픈' 개막이 이틀 앞으로 다가왔다. 지난해 우승자 8명 가운데 5명이 출사표를 던져 뜨거운 경쟁을 예고했다. 지난해 우승자 맹동섭은 이번 대회를 제패해 프로미오픈 2회 및 3년 연속 KPGA 코리안투어 우승을 달성한다는 각오다.model
 patrie 18일 포천 대유몽베르컨트리클럽에서 열리는 'DB손해보험 프로미오픈' 개막이 이틀 앞으로 다가왔다. 지난해 우승자 맹동섭은 이번 대회를 제패해 프로미오픈 2회 및 3년 연속 KPGA 코리안투어 우승을 달성한다는 각오다.
********************
error type2
OCN 토일 오리지널 '킬잇(Kill it)'이 킬러 김수현(장기용)의 과거와 형사 도현진(나나)이 파헤치고 있는 연쇄살인 사건의 내막을 막힘없이 차근차근 풀어내며 어느덧 반환점을 돌아섰다.user
********************
error type3
서종균 SH공사 주거복지처장은 임대주택의 유형이 계속 추가됐고, 지금도 새로운 하위 유형이 늘어나고 있다며 공급대상자인 시민들이 쉽게 알 수 있게 통합해야 한다고 강조했다.model
model
서종균 SH공사 주거복지처장은 "현 임대주택 정책의 문제점과 이에 따른 임대주택 유형통합의 필요성에 공감한다며 임대주택 수요자들이 임대주택 정책을 쉽게 이해하고 신청할 수 있는 유형통합을 위해 SH공사가 최선을 다하겠다"고 밝혔다.
********************
error type4
중기부 장관이 직원들에게 독서토론을 제안하며 새내기 장관으로서의 일정을 '조직 내부 소통'으로 시작했다.municipi=박영선 중소벤처기업부 장관이 9일 중기부 내부망을 통해 직원들에게 보낸 편지에서 "희망과 활력이 넘치는 강한 중소벤처기업부로 만들고 싶다"며 "함께 책을 읽고 이야기를 나누며 변화하는 세상에 대한 우리와 대한민국의 미래를 논하고자 한다"고 강조했다.
*****************

In [18]:
model1 = globals()[model_list[1]].copy()
print(model1.shape)

(5000, 4)


In [19]:
model1_error_delete = ['user',
                       r'\bmunicipi\b' #bmunicipi로 시작하는 단어들
                       ]

model1_error_split = [
                     'model',
                     'modelrehension',
                     'patrie:',
                     'centrifuga',
                     'modelrehension · 100자 제출.'
                     ]




In [20]:
for word in model1_error_delete:
    print('-------------------------------------------------------------\n\n')
    delete_words(model1, word)

-------------------------------------------------------------


@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
error_word : user
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
수정전 오류수:  62
수정전 문장 
:  '해치' 8~34회에서 경종이 사망한 이후, 이금이 대권을 쟁취하고 조선의 전성기를 이끈 왕 영조가 되어가는 과정을 본격적으로 그릴 것이다. 정일우는 이처럼 이금이 되어 이 모든 이야기를 시청자에게 보여줄 것이다.userumbersome한 극 중에서도 정일우의 장점이 극중 경종의 죽음 앞에서 또 다시 빛난다.model
model
'해치' 8~34회에서 경종이 사망한 이후, 이금이 대권을 쟁취하고 조선의 전성기를 이끈 왕 영조가 되어가는 과정을 본격적으로 그릴 것이다. 정일우는 이처럼 이금이 되어 이 모든 이야기를 시청자에게 보여줄 것이다.
*********************************************************************
수정후 오류수:  0
수정후 문장 
:  '해치' 8~34회에서 경종이 사망한 이후, 이금이 대권을 쟁취하고 조선의 전성기를 이끈 왕 영조가 되어가는 과정을 본격적으로 그릴 것이다. 정일우는 이처럼 이금이 되어 이 모든 이야기를 시청자에게 보여줄 것이다.umbersome한 극 중에서도 정일우의 장점이 극중 경종의 죽음 앞에서 또 다시 빛난다.model
model
'해치' 8~34회에서 경종이 사망한 이후, 이금이 대권을 쟁취하고 조선의 전성기를 이끈 왕 영조가 되어가는 과정을 본격적으로 그릴 것이다. 정일우는 이처럼 이금이 되어 이 모든 이야기를 시청자에게 보여줄 것이다.
-------------------------------------------------------

In [21]:
for word in model1_error_split:
    print('-------------------------------------------------------------\n\n')
    error_start(model1, word)

-------------------------------------------------------------


@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
error_word : model
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
수정전 오류수:  1394
수정전 문장 
:  5G 이동통신 서비스를 상용화한 것을 기념하는 '코리안 5G 테크-콘서트'가 8일 서울 올림픽공원 K아트홀에서 열렸다. 문재인 대통령, 홍남기 부총리 겸 기획재정부 장관과 유영민 과학기술정보통신부 장관 등 정부 관계자, 박정호 SK텔레콤 사장, 황창규 KT 회장, 하현회 LG유플러스 부회장, 고동진 삼성전자 IM(IT모바일) 부문장(사장), 정도현 LG전자 대표 등 300여 명이 참석한 가운데 5G 기술을 활용해 초고속 초저지연 초연결이라는 3대 특징 중 하나인 초저지연을 실감하게 했다.model
model
5G 이동통신 서비스를 상용화한 것을 기념하는 '코리안 5G 테크-콘서트'가 8일 서울 올림픽공원 K아트홀에서 열렸다. 문재인 대통령, 홍남기 부총리 겸 기획재정부 장관과 유영민 과학기술정보통신부 장관 등 정부 관계자, 박정호 SK텔레콤 사장, 황창규 KT 회장, 하현회 LG유플러스 부회장, 고동진 삼성전자 IM(IT모바일) 부문장(사장), 정도현 LG전자 대표 등 300여 명이 참석한 가운데 5G 기술을 활용해 초고속 초저지연 초연결이라는 3대 특징 중 하나인 초저지연을 실감하게 했다.
*********************************************************************
수정후 오류수:  0
수정후 문장 
:  5G 이동통신 서비스를 상용화한 것을 기념하는 '코리안 5G 테크-콘서트'가 8일 서울 올림픽공원 K아트홀에서 열렸다. 문재인 대통령, 홍남기 부총리 겸 기획재정부 장관과

In [22]:
model1['modify_summary'] = model1['modify_summary'].apply(lambda x: x if delete_nohangul(x) == 1 else ' ')

In [23]:
final_check(model1)

정상출력 : 3385
오류출력 : 1615
오류률 : 32.300000000000004 %


In [24]:
print('********************')
print('error type1') # patrie: -> 뒷부분 제거
print(model1['modify_summary'][870])
print('********************')
print('error type2') # user -> 제거
print(model1['modify_summary'][543])
print('********************')
print('error type3') # model -> 뒷부분 제거
print(model1['modify_summary'][75])
print('********************')
print('error type4') # municipi -> 잘못된 단어  / 해당 단어가 포함된 단어를 제거
print(model1['modify_summary'][97])
print('********************')
print('error type5') # centrifuga -> 잘못된 단어  / 오류 시작
print(model1['modify_summary'][885])
print('********************')
print('error type6') # modelrehension · 100자 제출. -> 잘못된 단어  / 오류 시작
print(model1['modify_summary'][152])

********************
error type1
올해 KPGA 코리안투어 개막전인 'DB손해보험 프로미오픈' 개막이 이틀 앞으로 다가왔다. 지난해 우승자 8명 가운데 5명이 출사표를 던져 뜨거운 경쟁을 예고했다. 지난해 우승자 맹동섭은 이번 대회를 제패해 프로미오픈 2회 및 3년 연속 KPGA 코리안투어 우승을 달성한다는 각오다.
********************
error type2
OCN 토일 오리지널 '킬잇(Kill it)'이 킬러 김수현(장기용)의 과거와 형사 도현진(나나)이 파헤치고 있는 연쇄살인 사건의 내막을 막힘없이 차근차근 풀어내며 어느덧 반환점을 돌아섰다.
********************
error type3
서종균 SH공사 주거복지처장은 임대주택의 유형이 계속 추가됐고, 지금도 새로운 하위 유형이 늘어나고 있다며 공급대상자인 시민들이 쉽게 알 수 있게 통합해야 한다고 강조했다.
********************
error type4
중기부 장관이 직원들에게 독서토론을 제안하며 새내기 장관으로서의 일정을 '조직 내부 소통'으로 시작했다.=박영선 중소벤처기업부 장관이 9일 중기부 내부망을 통해 직원들에게 보낸 편지에서 "희망과 활력이 넘치는 강한 중소벤처기업부로 만들고 싶다"며 "함께 책을 읽고 이야기를 나누며 변화하는 세상에 대한 우리와 대한민국의 미래를 논하고자 한다"고 강조했다.
********************
error type5
과학기술정보통신부는 지난해 말 미국에서 우주로 쏘아올린 '차세대소형위성 1호'가 안정화 단계에 들어섰다고 16일 발표했다.
********************
error type6
코오롱생명과학은 9일 오후 인보사와 관련된 핵심 쟁점을 정리한 자료를 발표했다. 코오롱생명과학은 지난달 31일 전격 판매가 중지됐다. 코오롱생명과학은 지난달 31일 전격 판매가 중지됐다. 코오롱생명과학은 9일 오후 인보사와 관련된 핵심 쟁점을 정리한 자료를 발표했다.


# model1_10000 오류항 점검

In [25]:
model2 = globals()[model_list[2]].copy()

print(model2.shape)

(5000, 4)


In [26]:

# -> 오류가 발생하면 특정 단어가 등장

print('********************')
print('error type1') # patrie:
print(model2['summary'][121])
print('********************')
print('error type2') # Româ
print(model2['summary'][294])
print('********************')
print('error type3') # model
print(model2['summary'][494])

# 저 단어들이 포함된 뒷부분은 제거

********************
error type1
지난해 중국을 제치고 7년 만에 국가별 수주 1위를 탈환한 한국 조선업계에 다시 '빨간불'이 켜졌다. 올해 1분기 중국은 전 세계에서 발주된 선박 231만CGT 중 절반을 웃도는 127만CGT(37척·55%)를 수주했다.
 patrie:김보형 기자 khy@kihoilbo.co.kr patrie:김보형 기자 khy@kihoilbo.co.kr patrie:김보형 기자 khy@kihoilbo.co.kr patrie:김보형 기자 khy@kihoilbo.co.kr patrie:김보형 기자 khy@kihoilbo.co.kr patrie:김보형 기자 khy@kihoilbo.co.kr patrie:김보형 기자 khy@kihoilbo.co.kr patrie:김보형 기자 khy@kihoilbo.co.kr patrie:김보형 기자 khy@kihoilbo.co.kr patrie:김보형 기자 khy@kihoilbo.co.kr patrie:김보형 기자 khy@kihoilbo.co.kr patrie:김보형 기자 khy@kihoilbo.co.kr patrie:김보형 기자 khy@kihoilbo.co.kr patrie:김보형 기자 khy@kihoilbo.co.kr patrie:김보형 기자 khy@kihoilbo.co.kr patrie:김보형 기자 khy@kihoilbo.co.kr patrie:김보형 기자 khy@kihoilbo.co.kr patrie:김보형 기자 khy@kihoilbo.co.kr patrie:김보형 기자 khy@kihoilbo.co.kr patrie:김보형 기자 khy@kihoilbo.co.kr patrie:김보형 기자 khy@kihoilbo.co.kr patrie:김보형 기자 khy@kihoilbo.co.kr patrie:김보형 기자 khy@kihoilbo.co.kr
********************
error type2
박유천(32)이 긴급 기자회견을 열어 "연예인 지인 A씨가 권유해서 하게 됐다"고 밝혔다.

In [27]:
model2_error_split = ['model',
                     'patrie:',
                     'Româ'
                     ]

for word in model2_error_split:
    print('-------------------------------------------------------------\n\n')
    error_start(model2, word)

-------------------------------------------------------------


@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
error_word : model
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
수정전 오류수:  58
수정전 문장 
:  울산시는 올해 2만3000여 개 일자리를 창출하기로 했다.
 patrie 울산의 실업률을 낮추기 위해 올해 24.7%인 7314억원을 일자리 및 경제활성화 분야 사업비로 편성했다. 울산=김형욱 기자 kjk@kihoilbo.co.kr 기호일보, KIHOILBO
model
울산시는 올해 2017년 1월 1일 이후 지역 중소기업에 취업했거나 창업 후 전입한 기준 중위소득 150% 이하인 300명을 대상으로 매월 10만원씩 열 차례 주거비 100만원을 지원하는 청년이 만드는 울산 프로젝트를 본격 시행하기로 했다.
*********************************************************************
수정후 오류수:  0
수정후 문장 
:  울산시는 올해 2만3000여 개 일자리를 창출하기로 했다.
 patrie 울산의 실업률을 낮추기 위해 올해 24.7%인 7314억원을 일자리 및 경제활성화 분야 사업비로 편성했다. 울산=김형욱 기자 kjk@kihoilbo.co.kr 기호일보, KIHOILBO
-------------------------------------------------------------


@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
error_word : patrie:
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
수정전 

In [28]:
model2['modify_summary'] = model2['modify_summary'].apply(lambda x: x if delete_nohangul(x) == 1 else ' ')

final_check(model2)

정상출력 : 4887
오류출력 : 113
오류률 : 2.26 %


# model3 - prompt10000

프롬프트를 대화형으로 변경한 모형



In [29]:
model3 = globals()[model_list[3]].copy()

print(model3.shape)

(5000, 4)


In [30]:

# -> 오류가 발생하면 특정 단어가 등장
# 'Provide a summary with about one sentences of one hundred characters for the following article'
# 그중에서 model로 시작하는 문장이 있으면 뒷부분만 요약본

model3_error_sentence = 'Provide a summary with about one sentences of one hundred characters for the following article'

model3['modify_summary'] = model3.apply(
    lambda x: 'error' if (model3_error_sentence in x['modify_summary']) and ('model' not in x['modify_summary'])
            else x['modify_summary'].split('model')[-1] if (model3_error_sentence in x['modify_summary']) and ('model' in x['modify_summary'])
                                                else x['modify_summary'], axis=1)


print('에러문장만')
print(model3['summary'][491])
print('********************')
print(model3['modify_summary'][491])

print('-------------------------------------')
print('에러문장만 + model')
print(model3['summary'][812])
print('********************')
print(model3['modify_summary'][812])

에러문장만
2016년 기준 세계 마약 사용자 수는 2억7500만 명으로 추산됐다. 2006년 2억800만 명에서 10년 새 32% 증가했다. 2016년 세계 마약 사용자 수는 2억7500만 명으로 추산됐다. 2006년 2억800만 명에서 10년 새 32% 증가했다.
assistant
    Provide a summary with about one sentences of one hundred characters for the following article.
    Synonymes: 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마
********************
error
-------------------------------------
에러문장만 + model
현대차그룹은 네이버, 카카오 출신 기술자들이 세운 스타트업(신생 벤처기업) 코드42에 전략적 투자를 진행했다. 현대차와 코드42는 스마트 모빌리티 솔루션을 제공하는 기업으로 거듭나야 한다고 강조하고 있다. 정의선 현대차그룹 총괄수석부회장은 평소 "현대·기아차가 앞으로 스마트 모빌리티 솔루션을 제공하는 기업으로 거듭나야 한

In [31]:
model3['modify_summary'] = model3['modify_summary'].apply(lambda x: x if delete_nohangul(x) == 1 else ' ')
final_check(model3)

정상출력 : 4584
오류출력 : 416
오류률 : 8.32 %


In [32]:
print('********************')
print('error type1') # patrie:
print(model3['summary'][491])
print('********************')
print('error type2') # Româ
print(model3['summary'][294])
print('********************')
print('error type3') # model
print(model3['summary'][494])

# 저 단어들이 포함된 뒷부분은 제거

********************
error type1
2016년 기준 세계 마약 사용자 수는 2억7500만 명으로 추산됐다. 2006년 2억800만 명에서 10년 새 32% 증가했다. 2016년 세계 마약 사용자 수는 2억7500만 명으로 추산됐다. 2006년 2억800만 명에서 10년 새 32% 증가했다.
assistant
    Provide a summary with about one sentences of one hundred characters for the following article.
    Synonymes: 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마약류, 마
********************
error type2
박유천(32)이 긴급 기자회견을 열어 "저는 결단코 마약을 하지 않았고 권유한 적은 더더욱 없다"고 밝혔다. 박유천은 10일 오후 서울 프레스센터에서 긴급 기자회견을 열어 "저는 결단코 마약을 하지 않았고 권유한 적은 더더욱 없다"고 밝혔다. 박유천은 "저는 마약을 한 적도 없고 권유한 적은 더더욱 없다"고 밝혔다. 박유천은 "저는 마약을 한 적도 없고 

In [33]:
model3_error_delete = [r'\bmunicipi\b' #bmunicipi로 시작하는 단어들
                       ]

model3_error_split = [
                     'împre',
                     'comuna:'

                     ]


In [34]:
for word in model3_error_delete:
    print('-------------------------------------------------------------\n\n')
    delete_words(model3, word)

-------------------------------------------------------------


@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
error_word : \bmunicipi\b
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
수정전 오류수:  183
수정전 문장 
:  남양유업 창업주 외손녀 황하나(31) 씨는 약 20만 명에 달하는 SNS 팔로워를 거느리던 셀럽에서 하루 아침에 마약 혐의 구속 피의자로 전락했다. 8일 경찰에 따르면 황씨는 지난 6일 구속 전 피의자 심문(영장실질심사)에서 '연예인 지인 A씨 권유로 마약을 계속하게 됐다'고 진술했다. 황씨는 이로 인해 2015년 필로폰을 투약 후 한동안 마약을 끊었다가 A씨 권유로 작년 말부터 다시 시작했다고 하고 있다. 경찰은 '자신은 (마약을) 그만하고 싶었지만, A 씨의 강요로 계속 투약할 수 밖에 없었다'고 말했다. 이에 따라 경찰은 황씨를 상대로 추가 조사를 진행하는 한편 조만간 A씨를 소환해 조사할 방침이다. 실제로 황씨는 그동안 사회관계망서비스(SNS)인 인스타그램에서 수많은 연예인들과 친분을 과시하곤 했다. 최근 '버닝썬 게이트' 빅뱅 승리의 카톡방 멤버로 알려진 최종훈과 이종현과의 인증샷을 SNS에 공개했다가 최근엔 삭제했다.municipi: 2009년 대마를 흡연해 2011년 기소유예 처분을 받은 전과가 있다. 또 지난 2016년 1월 대학생 조모씨 마약 사건에서 마약 유통 및 공급의 사실로 판결문에 등장하기도 했다. 하지만 그는 마약 투약 유통 보다 더 큰 처벌이 예상되는 마약 공급 혐의에 대해선 적극 부인하고 있다. 그렇다면 황씨가 마약 투약 혐의를 시인했지만 다시 투약하게 된 건 자신의 의지가 아니었다고 주장하는 것이 형량에는 어떤 영향을 미칠까.
*******************************

In [35]:
for word in model3_error_split:
    print('-------------------------------------------------------------\n\n')
    error_start(model3, word)

-------------------------------------------------------------


@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
error_word : împre
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
수정전 오류수:  136
수정전 문장 
:  KB자산운용은 코스닥 상장사 KMH에 이런 내용을 담은 주주서한을 최근 보냈다. KMH는 2013년부터 6년 동안 지배주주 순이익이 280% 늘었는데도 주가가 35% 오르는 데 그친 데 대해 복잡한 지배구조와 나빠진 현금흐름 때문이라고 분석했다. 이에 대해 KMH는 투자 주체로서의 역할을 보다 확실히 확립하겠다면서 사업 시너지와 현금 유동성을 고려해 투자 주체와 방식을 정하겠다고 답했다.
assistant
    Provide a summary with about one sentences of one hundred characters for the following article.
    Synonymes: SK와 알파벳이 신규 투자는 지주회사가 전담하고, 자회사는 본업에만 충실할 수 있도록 모범적인 지배구조를 짰다는 게 KB운용 평가다.
     împre. 임근호 기자
*********************************************************************
수정후 오류수:  0
수정후 문장 
:  
-------------------------------------------------------------


@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
error_word : comuna:
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

In [36]:
model3['modify_summary'] = model3['modify_summary'].apply(lambda x: x if delete_nohangul(x) == 1 else ' ')

final_check(model3)

정상출력 : 4418
오류출력 : 582
오류률 : 11.64 %


In [37]:
### model4 - model_2bnochat

In [38]:
model4 = globals()[model_list[4]].copy()

print(model4.shape)

(1000, 4)


In [39]:
model4['summary'][0]

'the summary of the news in korean:<end_of_turn>\n문재인 대통령은 8일 서울 올림픽공원에서 열린 5G플러스 전략발표에 참석해 5G가 대한민국 혁신성장의 인프라가 될 것이라며 5G가 4차 산업혁명 시대의 고속도로가 돼 새로운 기회를 열어 줄 것이라고 강조했다.<end_of_turn>\n<start_of_turn>'

In [40]:
model4['summary'][0].replace('the summary of the news in korean:<end_of_turn>\n', '').split('<end_of_turn>')[0]

'문재인 대통령은 8일 서울 올림픽공원에서 열린 5G플러스 전략발표에 참석해 5G가 대한민국 혁신성장의 인프라가 될 것이라며 5G가 4차 산업혁명 시대의 고속도로가 돼 새로운 기회를 열어 줄 것이라고 강조했다.'

In [41]:
model4['summary'] = model4['summary'].apply(lambda x: x.replace('the summary of the news in korean:<end_of_turn>\n', '').split('<end_of_turn>')[0])
model4['summary'] = model4['summary'].str.strip()
model4['modify_summary'] = model4['summary']

In [42]:
model4.to_excel('/content/drive/MyDrive/new_project/project2/test_result/model4.xlsx')

In [43]:
model4 = model4[0:1000]

In [56]:
model1.to_excel('model1.xlsx')

In [61]:
## 요약본 길이 계산하는 함수
def summary_len(model):
    len_total = 0
    text_cnt = 0
    for text in model['modify_summary']:
        if len(text) > 10:
            len_total += len(text)
            text_cnt += 1
    len_mean = len_total / text_cnt

    return len_mean

In [62]:
## valid 요약본 길이

origin_len_total = 0
for text in base_model['origin_summary']:
    origin_len_total += len(text)

origin_len = origin_len_total / len(base_model)
print('valid 요약본 길이', origin_len)

## 모델 통과한 요약본 길이

new_model_list = [f'model{i}' for i in range(5)]

for model in new_model_list:
    temp = globals()[model]
    print(f'{model} 요약본 길이 : ', summary_len(temp))

valid 요약본 길이 133.457
model0 요약본 길이 :  57.631968730713844
model1 요약본 길이 :  206.692
model2 요약본 길이 :  104.3626
model3 요약본 길이 :  168.96986936367466
model4 요약본 길이 :  159.905


# Calculate Rouge Score

In [46]:
df_valid = pd.read_csv('/content/drive/MyDrive/new_project/project2/traindata/문서요약 텍스트/kobart_valid.tsv', sep='\t')

df_valid.head()

,news,summary
0,"[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, 체...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...
1,] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다. 바른미래당이 8일 연 ...,8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...
2,[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀. 지난 3일 한국이 세계 최초로 ...,지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...
3,] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...
4,"[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를 ...",주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...


In [47]:
for i in range(5):
    temp = globals()[f'model{i}']
    temp = temp[['text', 'modify_summary']]
    temp.columns = ['news', 'modify_summary']
    globals()[f'df_model{i}']  = temp.merge(df_valid, on='news', how='left')


In [48]:
df_model0.head()

,news,modify_summary,summary
0,"[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, 체...",대한민국 5G 홍보대사를 자처한 문재인 대통령은 5G 시대가 막 시작됐다는 기대감을...,8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...
1,] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다. 바른미래당이 8일 연 ...,당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.,8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...
2,[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀. 지난 3일 한국이 세계 최초로 ...,"5G 기술을 활용한 원격 협연 방식으로 서울과 부산, 광주에서 동시에 하는 원격 협...",지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...
3,] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리...,박원순 서울시장은 '골목길 재생 시민 정책 대화'에서 서울시의 미래와 행복을 걱정하...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...
4,"[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를 ...",KB자산운용은 SK와 미국 알파벳의 간결한 지배구조를 평가하여 높은 수익률을 올리는...,주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...


In [49]:
df_model0

,news,modify_summary,summary
0,"[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, 체...",대한민국 5G 홍보대사를 자처한 문재인 대통령은 5G 시대가 막 시작됐다는 기대감을...,8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...
1,] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다. 바른미래당이 8일 연 ...,당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.,8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...
2,[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀. 지난 3일 한국이 세계 최초로 ...,"5G 기술을 활용한 원격 협연 방식으로 서울과 부산, 광주에서 동시에 하는 원격 협...",지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...
3,] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리...,박원순 서울시장은 '골목길 재생 시민 정책 대화'에서 서울시의 미래와 행복을 걱정하...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...
4,"[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를 ...",KB자산운용은 SK와 미국 알파벳의 간결한 지배구조를 평가하여 높은 수익률을 올리는...,주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...
...,...,...,...
4995,'캐릭터 라이선싱 페어 2019'가 17일부터 21일까지 서울 코엑스에서 열린다. ...,캐릭터 라이선싱 페어 2019는 한국 캐릭터 라이선싱 관련 기업과 단체 270여 곳...,17일~21일까지 서울 코엑스에서 개최되는 '캐릭터 라이선싱 페어 2019'에 '또...
4996,가천대 길병원(원장 김양우)은 영상의학과 컴퓨터단층촬영(CT) 장비를 재배치해 본관...,"가천대 길병원, 최신 CT 장비 도입, 영상 선명도 개선, 피폭량 감소, 재협착 유...","가천대 길병원은 CT 촬영 시간을 줄이고, 원스톱 검사를 구현하기 위해 CT 장비를..."
4997,KOTRA는 한국무역협회와 공동으로 이낙연 총리의 공식방문과 연계해 14일(현지시간...,KOTRA는 한국과 방글라데시 간 투자진출을 촉진하기 위해 방글라데시 투자청(BID...,"KOTRA는 지난 14일 방글라데시에서 기계장비, 소비재 등 유망시장 개척과 투자진..."
4998,'해커묵시록'으로 잘 알려진 최희원 작가가 3번째 책 '디지털에서 인생을 발견하다'...,해커묵시록은 첨단기술로 인해 우리의 일상이 어떻게 변화될 수 있는지와 위험성에 대한...,'해커묵시록'으로 잘 알려진 최희원 작가가 3번째 책 '디지털에서 인생을 발견하다'...


In [50]:
rouge = Rouge()

test_result = []

for model in new_model_list:
    temp = globals()[f'df_{model}']

    rogue1_recall_total = 0
    rogue1_precision_total = 0
    rogue1_f1_total = 0

    rogue2_recall_total = 0
    rogue2_precision_total = 0
    rogue2_f1_total = 0

    rogueL_recall_total = 0
    rogueL_precision_total = 0
    rogueL_f1_total = 0

    for i in range(len(temp)):
        origin_summary = temp['summary'][i]
        model_summary = temp['modify_summary'][i]

        test_score = rouge.get_scores(model_summary, origin_summary, avg=True)

        # test_score = rouge.get_scores(temp['modify_summary'][i], temp['summary'][i], avg=True)

        rogue1_recall_total += test_score['rouge-1']['r']
        rogue1_precision_total += test_score['rouge-1']['p']
        rogue1_f1_total += test_score['rouge-1']['f']

        rogue2_recall_total += test_score['rouge-2']['r']
        rogue2_precision_total += test_score['rouge-2']['p']
        rogue2_f1_total += test_score['rouge-2']['f']

        rogueL_recall_total += test_score['rouge-l']['r']
        rogueL_precision_total += test_score['rouge-l']['p']
        rogueL_f1_total += test_score['rouge-l']['f']

    rogue1_recall = rogue1_recall_total / len(temp)
    rogue1_precision = rogue1_precision_total / len(temp)
    rogue1_f1 = rogue1_f1_total / len(temp)

    rogue2_recall = rogue2_recall_total / len(temp)
    rogue2_precision = rogue2_precision_total / len(temp)
    rogue2_f1 = rogue2_f1_total / len(temp)

    rogueL_recall = rogueL_recall_total / len(temp)
    rogueL_precision = rogueL_precision_total / len(temp)
    rogueL_f1 = rogueL_f1_total / len(temp)

    test_result.append([model, rogue1_recall, rogue1_precision, rogue1_f1, rogue2_recall, rogue2_precision, rogue2_f1, rogueL_recall, rogueL_precision, rogueL_f1])


In [51]:
df_results = pd.DataFrame(test_result)
df_results.columns = ['model', 'rouge1_recall', 'rouge1_precision', 'rouge1_f1', 'rouge2_recall', 'rouge2_precision', 'rouge2_f1', 'rougeL_recall', 'rougeL_precision', 'rougeL_f1']
df_results

,model,rouge1_recall,rouge1_precision,rouge1_f1,rouge2_recall,rouge2_precision,rouge2_f1,rougeL_recall,rougeL_precision,rougeL_f1
0,model0,0.138808,0.325568,0.188009,0.062933,0.158355,0.086814,0.128845,0.303756,0.174810
1,model1,0.254604,0.284228,0.250201,0.136521,0.158857,0.135749,0.240713,0.268117,0.236139
2,model2,0.217403,0.363004,0.260531,0.118543,0.210587,0.144856,0.202074,0.338895,0.242568
3,model3,0.243286,0.314604,0.253099,0.136381,0.183115,0.143338,0.230416,0.297816,0.239468
4,model4,0.231991,0.258142,0.231527,0.115948,0.134401,0.117332,0.216128,0.240767,0.215770


In [52]:
df_results.to_excel('/content/drive/MyDrive/new_project/project2/test_result/rouge_score.xlsx', index = False)

NameError: name 'model0_error_rate' is not defined